In [41]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
#visualizations
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

#algorithms
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

#score metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [73]:
df = pd.read_csv(r'C:\Users\SHREYAN\Documents\ICPSR_06693\chronic disease Identification\liverfailure.csv')

In [74]:
df = df.dropna(axis = 0, subset=['ALF'])
df = df[['Age','Gender','Region','Weight','Height','Body Mass Index','Obesity','Waist',
         'Maximum Blood Pressure','Minimum Blood Pressure','Good Cholesterol','Bad Cholesterol',
         'Total Cholesterol','Dyslipidemia','PVD','ALF']]
total_missingvalues = df.isnull().sum()
total_missingvalues

Age                         0
Gender                      0
Region                      0
Weight                    133
Height                    139
Body Mass Index           206
Obesity                   206
Waist                     215
Maximum Blood Pressure    206
Minimum Blood Pressure    252
Good Cholesterol            8
Bad Cholesterol             8
Total Cholesterol           6
Dyslipidemia                0
PVD                         0
ALF                         0
dtype: int64

In [75]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan,strategy = 'median')
imputer = imputer.fit(df.iloc[:,3:13]) #SELECTING THE COLUMN WITH MISSING VALUES
df.iloc[:,3:13] = imputer.transform(df.iloc[:,3:13])

In [76]:
df=df.loc[df["ALF"]==0].sample(n=len(df.loc[df["ALF"]==1])).append(df.loc[df["ALF"]==1]) 
#balancing the dataset to have equal no of ALF=0 and ALF=1
y = df['ALF']
df = df.drop(['ALF','Region'],axis=1)
df

,Age,Gender,Weight,Height,Body Mass Index,Obesity,Waist,Maximum Blood Pressure,Minimum Blood Pressure,Good Cholesterol,Bad Cholesterol,Total Cholesterol,Dyslipidemia,PVD
4090,82,F,98.20,173.6,32.58,1.0,118.9,140.0,65.0,37.0,280.0,317.0,0,0
1985,67,F,93.10,179.1,29.02,0.0,108.3,184.0,92.0,37.0,153.0,190.0,0,0
5971,46,F,134.40,191.5,36.65,1.0,132.2,116.0,86.0,36.0,182.0,218.0,0,0
946,21,F,52.00,163.0,19.57,0.0,70.0,110.0,41.0,54.0,172.0,226.0,0,0
4355,21,F,84.30,198.0,21.50,0.0,81.0,112.0,62.0,52.0,97.0,149.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5979,73,M,62.80,150.8,27.62,0.0,89.0,154.0,65.0,48.0,141.0,189.0,0,1
5984,58,M,85.14,160.6,33.01,1.0,113.1,135.0,79.0,39.0,106.0,145.0,0,0
5985,69,F,83.50,183.5,24.80,0.0,100.0,141.0,72.0,52.0,206.0,258.0,1,0
5988,61,M,66.00,152.7,28.31,0.0,77.0,132.0,81.0,46.0,162.0,208.0,0,0


In [77]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X = LabelEncoder()
df.iloc[:,1] = labelencoder_X.fit_transform(df.iloc[:,1]) #SELECTING THE COLUMN WITH OBJECT TYPE

X = df
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
df

,Age,Gender,Weight,Height,Body Mass Index,Obesity,Waist,Maximum Blood Pressure,Minimum Blood Pressure,Good Cholesterol,Bad Cholesterol,Total Cholesterol,Dyslipidemia,PVD
4090,82,0,98.20,173.6,32.58,1.0,118.9,140.0,65.0,37.0,280.0,317.0,0,0
1985,67,0,93.10,179.1,29.02,0.0,108.3,184.0,92.0,37.0,153.0,190.0,0,0
5971,46,0,134.40,191.5,36.65,1.0,132.2,116.0,86.0,36.0,182.0,218.0,0,0
946,21,0,52.00,163.0,19.57,0.0,70.0,110.0,41.0,54.0,172.0,226.0,0,0
4355,21,0,84.30,198.0,21.50,0.0,81.0,112.0,62.0,52.0,97.0,149.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5979,73,1,62.80,150.8,27.62,0.0,89.0,154.0,65.0,48.0,141.0,189.0,0,1
5984,58,1,85.14,160.6,33.01,1.0,113.1,135.0,79.0,39.0,106.0,145.0,0,0
5985,69,0,83.50,183.5,24.80,0.0,100.0,141.0,72.0,52.0,206.0,258.0,1,0
5988,61,1,66.00,152.7,28.31,0.0,77.0,132.0,81.0,46.0,162.0,208.0,0,0


In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify = y)

In [83]:
from keras import Sequential
from keras.layers import Dense

In [84]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal', input_dim=X_train.shape[1]))
#Second  Hidden Layer
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [85]:
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [86]:
classifier.fit(X_train,y_train, batch_size=10, epochs=10)

Epoch 1/10
70/70 [==============================] - 3s 3ms/step - loss: 0.6929 - accuracy: 0.5489
Epoch 2/10
70/70 [==============================] - 0s 2ms/step - loss: 0.6884 - accuracy: 0.7328
Epoch 3/10
70/70 [==============================] - 0s 2ms/step - loss: 0.6623 - accuracy: 0.7759
Epoch 4/10
70/70 [==============================] - 0s 5ms/step - loss: 0.6024 - accuracy: 0.7830
Epoch 5/10
70/70 [==============================] - 0s 2ms/step - loss: 0.5348 - accuracy: 0.7859
Epoch 6/10
70/70 [==============================] - 0s 2ms/step - loss: 0.4916 - accuracy: 0.7960
Epoch 7/10
70/70 [==============================] - 0s 2ms/step - loss: 0.4702 - accuracy: 0.7931
Epoch 8/10
70/70 [==============================] - 0s 2ms/step - loss: 0.4601 - accuracy: 0.7888
Epoch 9/10
70/70 [==============================] - 0s 2ms/step - loss: 0.4540 - accuracy: 0.7931
Epoch 10/10
70/70 [==============================] - 0s 2ms/step - loss: 0.4505 - accuracy: 0.7974


In [87]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [90]:
xgb = XGBClassifier(random_state=10)

In [91]:
xgb.fit(X_train,y_train)

C:\Users\SHREYAN\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:44:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [92]:
pred = xgb.predict(X_test)

In [94]:
error = count/len(pred)
print( "Error for XGBoost= %f " % (error*100) + '%' )
accuracy = (1-error)
print( "Accuracy for XGBoost = %f " % (accuracy*100) + '%' )

Error for XGBoost= 20.258621 %
Accuracy for XGBoost = 79.741379 %


In [95]:
rf = RandomForestClassifier(random_state=10)
rf.fit(X_train,y_train)
pred_rf = rf.predict(X_test)
count = 0
for i in range( len(y_test) ):
    if pred_rf[i] != y_test.iloc[i]: 
        count = count + 1
error = count/len(pred_rf)
print( "Error for RF = %f " % (error*100) + '%' )
accuracy = (1-error)
print( "Accuracy for RF = %f " % (accuracy*100) + '%' )

Error for RF = 19.396552 %
Accuracy for RF = 80.603448 %


In [97]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5,],
    'criterion' :['gini', 'entropy']
}

In [98]:
from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 2)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=10),
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [4, 5],
                         'max_features': ['auto', 'sqrt'],
                         'n_estimators': [200, 500]})

In [104]:
best_param = CV_rfc.best_params_
best_param

{'criterion': 'gini',
 'max_depth': 4,
 'max_features': 'auto',
 'n_estimators': 500}

In [105]:
rf = RandomForestClassifier(**best_param)

In [107]:
rf.fit(X_train,y_train)
pred_rf = rf.predict(X_test)
count = 0
for i in range( len(y_test) ):
    if pred_rf[i] != y_test.iloc[i]: 
        count = count + 1
error = count/len(pred_rf)
print( "Error for RF = %f " % (error*100) + '%' )
accuracy = (1-error)
print( "Accuracy for RF = %f " % (accuracy*100) + '%')

Error for RF = 20.689655 %
Accuracy for RF = 79.310345 %
